In [ ]:
import piplite
await piplite.install('seaborn')
await piplite.install('Jinja2')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


In [ ]:
df = pd.read_csv("Campaign Reports.csv")

In [ ]:
df.head()

In [ ]:
df=df.drop(['id','Campaign title','Audience','Send Day','Total Revenue'], axis=1)
df['Send Date'] = pd.to_datetime(df['Send Date'], format='%m/%d/%Y')
df['Subject'] = df['Subject'].astype(str) 

df['Successful Deliveries'] = df['Total Recipients'] - df['Total Bounces']
df['Open Rate'] = df['Unique Opens'] / df['Successful Deliveries']
df['Click to Open Rate'] = df['Unique Clicks'] / df['Unique Opens']
df['Length'] = df['Subject'].apply(len)
df['No of Words'] = df['Subject'].apply(lambda x: len(x.split()))

df['Day of Week']= df['Send Date'].dt.day_name() 
df['Month'] = df['Send Date'].dt.month
df['Day'] = df['Send Date'].dt.day
df['Year'] = df['Send Date'].dt.year

In [ ]:
df.head()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('fivethirtyeight')

sns.distplot(df['Open Rate'], color = 'red')
plt.title('Open Rate Distribution', fontsize = 20)
plt.show()

In [ ]:
df.groupby('Day of Week')[['Open Rate', 'Click Rate','Click to Open Rate']].agg('mean').plot(
    kind='bar', figsize = (10,6), alpha=0.4, rot=0, cmap='copper');
plt.title("MailChimp G&F Metrics vs. Industry")

#Open Rate Benchmarks:
plt.scatter([-0.17,0.83,1.83,2.83,3.83,4.83,5.83],[0.216,0.22,0.205,0.203,0.217,0.218,0.218], c='black', marker='_');

#Click Rate Benchmarks:
plt.scatter([0,1,2,3,4,5,6], [0.022,0.023,0.021,0.021,0.023,0.024,0.023], c='brown', marker='_');

#Click-to-open Rate Benchmarks:
plt.scatter([0.17,1.17,2.17,3.17,4.17,5.17,6.17], [0.101, 0.106, 0.101, 0.101, 0.107,0.108,0.107], c='darkorange', marker='_');

df.groupby('Day of Week', sort='false')['Send Date'].agg('count').plot(kind='barh', figsize=(16, 4), color = 'blue')
plt.title("MailChimp G&F Newsletters Sent by Day")
plt.ylabel('')
plt.xlabel('')
plt.grid()
plt.show()


In [ ]:
df.groupby('Day of Week', sort='false')['Send Date'].agg('count')

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('fivethirtyeight')

sns.lineplot(x=df['Month'], y=df['Open Rate'], color = 'red');
plt.title('MailChimp Open Rate vs. Month', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('fivethirtyeight')

sns.lineplot(x=df['Month'], y=df['Click Rate'], color = 'red');
plt.title('MailChimp Click Rate vs. Month', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('fivethirtyeight')

sns.lineplot(x=df['Day of Week'], y=df['Open Rate'], color = 'black');
plt.title('MailChimp Open Rate vs. Day of the Week', fontsize = 20)
plt.show()


In [ ]:

plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('fivethirtyeight')

sns.lineplot(x=df['Day of Week'], y=df['Click Rate'], color = 'black')
plt.title('MailChimp Click Rate vs. Day of the Week', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('fivethirtyeight')

sns.scatterplot(x=df['Length'], y=df['Open Rate'], color = 'red')
sns.lineplot(x=df['Length'], y=df['Open Rate'], color = 'black')
plt.title('Open Rate vs Email Subject Line Length', fontsize = 20)
plt.show()


In [ ]:
df[['Subject','Open Rate', 'Length']].sort_values(['Open Rate'],ascending = False).style.background_gradient(cmap = 'bone')


In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('fivethirtyeight')

sns.scatterplot(x=df['No of Words'], y=df['Open Rate'], color = 'red')
sns.lineplot(x=df['No of Words'], y=df['Open Rate'], color = 'black')
plt.title('Open Rate vs to the Subject Line Number of Words', fontsize = 20)
plt.show()

In [ ]:
df[['Subject','Open Rate', 'No of Words']].sort_values(['Open Rate'],ascending = False).style.background_gradient(cmap = 'bone')


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words = 'english')
words = cv.fit_transform(df.Subject)
sum_words = words.sum(axis=0)
words_freq_x = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq_x = sorted(words_freq_x, key = lambda x: x[1], reverse = True)
frequency = pd.DataFrame(words_freq_x, columns=['word', 'freq'])
frequency.head(20)

In [ ]:
plt.style.use('fivethirtyeight')
color = plt.cm.Wistia(np.linspace(0, 1, 20))
frequency.head(20).plot(x='word', y='freq', kind='bar', figsize=(16, 4), color = color)
plt.title("Most Frequently Occurring Words in Email Subjects")
plt.ylabel('Count of the Words')
plt.xlabel('Words from Email Subjects')
plt.grid()
plt.show()